<a href="https://colab.research.google.com/github/SrijaniChakrabarti/Prediction-System-/blob/master/networkanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy
import pandas as pd
from queue import PriorityQueue


In [2]:
# Link travel time = free flow time * ( 1 + B * (flow/capacity)^Power ).
# Link generalized cost = Link travel time + toll_factor * toll + distance_factor * distance
# The network files also contain a "speed" value for each link. In some cases the "speed" values are consistent with the free flow times, in other cases they represent posted speed limits, and in some cases there is no clear knowledge about their meaning. All of the results reported below are based only on free flow travel times as described by the functions above, and do not use the speed values.

In [3]:
netfile = '/content/sample_data/Chicagosketchnet.tntp'
net = pd.read_csv(netfile, skiprows=8, sep='\t')

trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed

# And drop the silly first andlast columns
net.drop(['~', ';'], axis=1, inplace=True)


In [4]:
net

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,547,49500,0.86267,0.00,0.15,4,0,0,3
1,2,548,49500,0.86267,0.00,0.15,4,0,0,3
2,3,549,49500,0.86267,0.00,0.15,4,0,0,3
3,4,550,49500,0.86267,0.00,0.15,4,0,0,3
4,5,551,49500,0.86267,0.00,0.15,4,0,0,3
...,...,...,...,...,...,...,...,...,...,...
2945,931,906,500,22.65430,14.78,0.15,4,0,0,1
2946,932,386,49500,0.86267,0.00,0.15,4,0,0,3
2947,932,515,5000,10.44260,5.93,0.15,4,0,0,2
2948,933,387,49500,0.86267,0.00,0.15,4,0,0,3


In [5]:
net.head()

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,547,49500,0.86267,0.0,0.15,4,0,0,3
1,2,548,49500,0.86267,0.0,0.15,4,0,0,3
2,3,549,49500,0.86267,0.0,0.15,4,0,0,3
3,4,550,49500,0.86267,0.0,0.15,4,0,0,3
4,5,551,49500,0.86267,0.0,0.15,4,0,0,3


In [6]:
net.columns

Index(['init_node', 'term_node', 'capacity', 'length', 'free_flow_time', 'b',
       'power', 'speed', 'toll', 'link_type'],
      dtype='object')

In [7]:
len(net)


2950

In [8]:
net.columns[0]

'init_node'

In [17]:
# taking all edges of graph in list
init_node_list = list(net.init_node)
term_node_list = list(net.term_node)
length_list = list(net.length)
t = len(net)
edge_list = []
i = 0
while i < t:
  term_temp = term_node_list[i]
  init_temp = init_node_list[i]
  l_temp = length_list[i]
  edge_list.append((init_temp,term_temp,l_temp))
  i += 1


In [18]:
# Functions for constructing adjacency list
adj_list = {}
mylist = []
def add_node(node):
  if node not in mylist:
    mylist.append(node)
  else:
    print("Node ",node," already exists!")
 
def add_edge(node1, node2, dist):
  temp = []
  
  if node1 in mylist and node2 in mylist:
    if node1 not in adj_list:
      temp.append((node2,dist))
      adj_list[node1] = temp
   
    elif node1 in adj_list:
      temp.extend(adj_list[node1])
      temp.append((node2,dist))
      adj_list[node1] = temp
       
  else:
    print("Nodes don't exist!")
  



      

In [19]:
# giving all the inputs for constructing adjacency list
for i in init_node_list:
  add_node(i)
for l in term_node_list:
  add_node(l)
for (ini,term,dist) in edge_list:
  add_edge(ini,term,dist)


Node  388  already exists!
Node  388  already exists!
Node  388  already exists!
Node  389  already exists!
Node  389  already exists!
Node  390  already exists!
Node  390  already exists!
Node  391  already exists!
Node  391  already exists!
Node  391  already exists!
Node  392  already exists!
Node  392  already exists!
Node  392  already exists!
Node  393  already exists!
Node  393  already exists!
Node  393  already exists!
Node  394  already exists!
Node  394  already exists!
Node  394  already exists!
Node  395  already exists!
Node  395  already exists!
Node  395  already exists!
Node  396  already exists!
Node  397  already exists!
Node  397  already exists!
Node  397  already exists!
Node  398  already exists!
Node  398  already exists!
Node  398  already exists!
Node  399  already exists!
Node  399  already exists!
Node  399  already exists!
Node  400  already exists!
Node  400  already exists!
Node  400  already exists!
Node  401  already exists!
Node  401  already exists!
N

In [15]:
len(mylist) # mylist is list of all nodes without duplicates

933

In [56]:
adj_list # dictionary for tracking adjacent nodes

{1: [(547, 0.86267)],
 2: [(548, 0.86267)],
 3: [(549, 0.86267)],
 4: [(550, 0.86267)],
 5: [(551, 0.86267)],
 6: [(552, 0.86267)],
 7: [(553, 0.86267)],
 8: [(554, 0.86267)],
 9: [(555, 0.86267)],
 10: [(556, 0.86267)],
 11: [(557, 0.86267)],
 12: [(558, 0.86267)],
 13: [(559, 0.86267)],
 14: [(560, 0.86267)],
 15: [(561, 0.86267)],
 16: [(562, 0.86267)],
 17: [(563, 0.86267)],
 18: [(564, 0.86267)],
 19: [(565, 0.86267)],
 20: [(566, 0.86267)],
 21: [(567, 0.86267)],
 22: [(568, 0.86267)],
 23: [(569, 0.86267)],
 24: [(570, 0.86267)],
 25: [(571, 0.86267)],
 26: [(572, 0.86267)],
 27: [(573, 0.86267)],
 28: [(574, 0.86267)],
 29: [(575, 0.86267)],
 30: [(576, 0.86267)],
 31: [(577, 0.86267)],
 32: [(578, 0.86267)],
 33: [(579, 0.86267)],
 34: [(580, 0.86267)],
 35: [(581, 0.86267)],
 36: [(582, 0.86267)],
 37: [(583, 0.86267)],
 38: [(584, 0.86267)],
 39: [(585, 0.86267)],
 40: [(586, 0.86267)],
 41: [(587, 0.86267)],
 42: [(588, 0.86267)],
 43: [(589, 0.86267)],
 44: [(590, 0.86267)

In [59]:
def make_path(parent, dest_node):
    if dest_node not in parent:
        return None
    v = dest_node
    path = []
    while v is not None: # root has null parent
        path.append(v)
        v = parent[v]
    return path #::-1

def return_distance(node1,node2,G):
  for (node2,dis) in G[node1]:
    return dis

def Calculate_sumofShortestpathdistancevalue(shortestpath_vertices_list,G):
  t = len(shortestpath_vertices_list)
  sum = 0
  i = 0
  while(i < t):
    x = i + 1
    if ( x == t):
      break
    else:
      sum += return_distance(shortestpath_vertices_list[i],shortestpath_vertices_list[x],G)
    i = i + 1
  print(sum)



def dijkstra(G, start_node, dest_node):
    
    visited = set()
    cost = {start_node: 0}
    parent = {start_node: None}
    prq = PriorityQueue()
    least_length = 0
    prq.put((0, start_node))
    while prq:
        while not prq.empty():
            _, vertex = prq.get() # finds lowest cost vertex
            # loop until we get a fresh vertex
            if vertex not in visited: break
        else: # if todo ran out
            break # quit main loop
        visited.add(vertex)
        if vertex == dest_node:
            break
        for (neighbor, distance) in G[vertex]:
            if neighbor in visited: continue # skip these to save time
            old_cost = cost.get(neighbor, float('inf')) # default to infinity
            new_cost = cost[vertex] + distance
            if new_cost < old_cost:
                prq.put((new_cost, neighbor))
                cost[neighbor] = new_cost
                parent[neighbor] = vertex

    vert_list = make_path(parent,dest_node)
    reverse_vert_list = vert_list[::-1]
                
    return Calculate_sumofShortestpathdistancevalue(reverse_vert_list,G)




26.686080000000015


In [65]:
rv

In [31]:
paren = dijkstra(adj_list,933,417)
temp = make_path(paren,417)
reverse_temp = temp[::-1]

In [36]:
def return_distance(node1,node2,G):
  for (node2,dis) in G[node1]:
    return dis


In [42]:
def Calculate_sumofShortestpathdistancevalue(shortestpath_vertices_list):
  t = len(shortestpath_vertices_list)
  sum = 0
  i = 0
  while(i < t):
    x = i + 1
    if ( x == t):
      break
    else:
      sum += return_distance(shortestpath_vertices_list[i],shortestpath_vertices_list[x],adj_list)
  i = i + 1
  return sum


In [39]:
sum

44.578760000000024